# Introduction
In this notebook we demonstrate some useful ways LLM's can be employed beyond simple question and answering tasks. We will show how to use LLMs to:

+ write api calls to trigger other software (tool calls)
+ break down multi-step problems into multiple smaller steps (goal-decomposition)
+ categorize an input to a set of pre-defined labels.

Finally, we will combine these features to create a simple AI agent capable of autonomously tackling multi-step problems.

# Installation
We have already set up a container with the required depencies for you to run everything in this module.  I've included dependency installation instructions here in case you want to get this notebook running on your own setup.  You will need to run a local ollama server (https://ollama.com/) for LLM inference and install the ollama python package (https://github.com/ollama/ollama-python).  Note that the pip install of the ollama package below only includes a python integration for ollama, you still need to download and install the ollama backend from their website.

1) Install ollama:

    pip install ollama

2) Start an ollama server by running the command below in a terminal. Note, there's a method below to run the server in the background of this notebook but at home I'd recommend running the server in a separate terminal and not in a notebook so you can see the server status more easily.

    ollama serve

Ollama should download models automatically whenever they are requested, but to force it to get the model I'm using below you can always run the following command in a new terminal once the server is up.

    ollama run llama3.2

Not all models are able to accept tool calls, when browsing the models library (https://ollama.com/library), the models with the **tools** icon under their name will accept tool calls.

You will also need the duckduckgo search python library to allow our agent to browse the web

    pip install -U duckduckgo_search

# AI Agent Tools
### Tool Transcriber
This function converts a function's python code into a dictionary object that contains a description of what the function does and defines its inputs and outputs so that the LLM will understand how to execute it. This code is copied from here https://github.com/meirm/ollama-tools.

When querying the LLM to write a tool call we will use the function **llm_prompt_tool()** which requires a prompt as well as a list of the available tools the model can use. We can use any python function we want as a tool. To create this list of tools to pass to the **llm_prompt_tool()** function, you just need to run 

    tools = [generate_function_description(<function 1 name>),
             generate_function_description(<function 2 name>),
             ...
             ]

You then use this tools object to prompt the LLM like

    tool_output = llm_prompt_tool(prompt='your prompt', tools=tools)

This would send the LLM your prompt and list of tools, the LLM will respond with the tool it wants to execute and the input arguments, and then we execute the tool function and return the function's output. 

In [1]:
import inspect
import json
import re
def generate_function_description(func):
    func_name = func.__name__
    docstring = func.__doc__

    # Get function signature
    sig = inspect.signature(func)
    params = sig.parameters

    # Create the properties for parameters
    properties = {}
    required = []

    # Process the docstring to extract argument descriptions
    arg_descriptions = {}
    if docstring:
        # remove leading/trailing whitespace or leading empty lines and split into lines
        docstring = re.sub(r'^\s*|\s*$', '', docstring, flags=re.MULTILINE)
        lines = docstring.split('\n')
        current_arg = None
        for line in lines:
            line = line.strip()
            if line:
                if ':' in line:
                    # strip leading/trailing whitespace and split into two parts
                    line = re.sub(r'^\s*|\s*$', '', line)
                    parts = line.split(':', 1)
                    if parts[0] in params:
                        current_arg = parts[0]
                        arg_descriptions[current_arg] = parts[1].strip()
                elif current_arg:
                    arg_descriptions[current_arg] += ' ' + line.strip()

    for param_name, param in params.items():
        param_type = 'string'  # Default type; adjust as needed based on annotations
        if param.annotation != inspect.Parameter.empty:
            param_type = param.annotation.__name__.lower()

        param_description = arg_descriptions.get(param_name, f'The name of the {param_name}')

        properties[param_name] = {
            'type': param_type,
            'description': param_description,
        }
        if param.default == inspect.Parameter.empty:
            required.append(param_name)

    # Create the JSON object
    function_description = {
        'type': 'function',
        'function': {
            'name': func_name,
            'description': docstring.split('\n')[0] if docstring else f'Function {func_name}',
            'parameters': {
                'type': 'object',
                'properties': properties,
                'required': required,
            },
        },
    }

    return function_description

### Tool functions
These are the functions that we'll set up as tools for the AI agent. Note that each function definition below has a multi-line docstring that describes its purpose, inputs, and outputs.  These are what **generate_function_description()** uses to make our tool description for the LLM call.

In [2]:
import json
import requests
import time
from duckduckgo_search import DDGS


def get_duckduckgo_result(query: str) -> str:
    """
    Get the top DuckDuckGo search result for the given query.
    query: The search query to send to DuckDuckGo.
    """
    results = DDGS().text(query, 
                          region='wt-wt', 
                          safesearch='off', 
                          timelimit='y', 
                          max_results=10)
    
    return results[0]['body']

def do_math(a:int, op:str, b:int)->list:
    """
    Performs math on the inputs
    a: The first operand
    op: The operation to perform (one of '+', '-', '*', '/')
    b: The second operand
    """
    res = "Nan"
    if op == "+":
        res = str(int(a) + int(b))
    elif op == "-":
        res = str(int(a) - int(b))
    elif op == "*":
        res = str(int(a) * int(b))
    elif op == "/":
        if int(b) != 0:
            res = str(int(a) / int(b))
    return res

def get_current_time() -> str:
    """Get the current time"""
    current_time = time.strftime("%H:%M:%S")
    return f"The current time is {current_time}"

def get_current_weather(city:str) -> str:
    """Get the current weather for a city
    Args:
        city: The city to get the weather for
    """
    base_url = f"http://wttr.in/{city}?format=j1"
    response = requests.get(base_url)
    data = response.json()
    return f"The current temperature in {city} is: {data['current_condition'][0]['temp_C']}°C"

### Building tools object example
Here is an example that builds the stringified tools data to send to our LLM

In [3]:
tools = [
    generate_function_description(get_duckduckgo_result),
    generate_function_description(do_math)]

print(f"Stringified tool descriptions:\n{json.dumps(tools, indent=4)}")

Stringified tool descriptions:
[
    {
        "type": "function",
        "function": {
            "name": "get_duckduckgo_result",
            "description": "Get the top DuckDuckGo search result for the given query.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "str",
                        "description": "The search query to send to DuckDuckGo."
                    }
                },
                "required": [
                    "query"
                ]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "do_math",
            "description": "Performs math on the inputs",
            "parameters": {
                "type": "object",
                "properties": {
                    "a": {
                        "type": "int",
                        "description": "The first operand"
                    }

# LLM Functions

### LLM Function: llm_prompt(prompt, system_message, seed) -> str
This function sends the prompt to the LLM and returns the response text

In [4]:
import ollama

# Sends the prompt to the LLM and returns the message response string
def llm_prompt(prompt: str, 
               system_message: str="You are a helpful AI Agent named Kiwi.", 
               seed: int=-1, 
               model: str="llama3.2") -> str:

        # generate a text response by sending our prompt to the ollama server 
        response = ollama.chat(
            model="llama3.2", 
            options={"seed":seed}, 
            messages=[
            {"role": "system", "content": system_message},{"role": "user", "content": prompt}],
        )
        return response['message']['content']

### LLM Function: llm_prompt_tool(prompt, tools, system_message, seed) -> str
This function sends the prompt and a list of tools to the LLM, executes the tool call the LLM generates and returns the result. Note, it only executes the first tool call the LLM wants to make.

In [5]:
import ollama

# Sends the prompt to the LLM and returns the message response string
def llm_prompt_tool(prompt: str, 
               tools: list,
               system_message: str="You are a helpful AI Agent named Kiwi.", 
               seed: int=-1, 
               model: str="llama3.2") -> str:

        response = ollama.chat(
            model="llama3.2", 
            options={"seed":seed}, 
            messages=[
            {"role": "system", "content": system_message},{"role": "user", "content": prompt}],
            tools=tools
        )

        # pull out the tool call dictionary from the response object
        tools_calls = response['message']['tool_calls']
        print(f"Tool call:\n{tools_calls}\n") 
    
        # get the name and input arguments of the tool
        tool_name = tools_calls[0]['function']['name']
        arguments = tools_calls[0]['function']['arguments']

        # execute the tool
        result = globals()[tool_name](**arguments)
    
        return result

### LLM Function: llm_create_list(prompt) -> dict
This function queries the LLM with prompt and has it return a json-formatted list with descriptions of each list item

In [6]:
# formats the prompt into a template string that asks for list in response
def create_list_template(prompt: str) -> str:

    list_schema = """```json
{
"list_description": "Describe list contents here",
"content":[
{"name": "Name of list item 1", "description": "Description of list item 1"},
{"name": "Name of list item 2", "description": "Description of list item 2"},
...
]
}
```"""
    
    output_prompt = f"""The following prompt requires you to respond with a list.

*Prompt:* {prompt}

*List format schema:*
{list_schema}

The "..." indicates that the list can be as many items long as you require.

Respond with the list that address the prompt using the above formatting schema."""
    
    return output_prompt

# Given a prompt, ask the LLM to make a list in response, retry until it responds in a good format
# Will respond with a dictionary with a list 'description' and 'content' which is an array of dicts
# with 'name' and 'description' of each list item.  An empty dict is returned if it fails to make
# a well formatted list
def llm_create_list(prompt: str) -> dict:

    # Desired system message
    system_message = "You are a helpful AI Agent named Kiwi."

    # Embed our prompt into the pick_option_template
    prompt = create_list_template(prompt)
    
    # # For testing, print templated prompt
    # print(f"Templated Prompt:\n{prompt}")

    # set the choice to no choice selected
    generated_list = {}
    
    # We'll try 10 times to get a valid choice
    for seed in range(1, 10):

        # # For testing, print current seed value
        # print(f"Querying model with random seed vaule {seed}...\n")

        # print progress
        status = f"trying to create list [{seed}/10] times..." 
        print(status,end='')
        
        # Send our pick tools prompt to the model
        response_text = llm_prompt(prompt, system_message)
        
        # try to convert the LLM response to an int
        try:

            # find the json data in the response which should start with ```json and end with ```
            start_index = response_text.find("```json")+7
            end_index = response_text.find("```", start_index)

            # pull out just the json data
            cleaned_list_string = response_text[start_index:end_index]

            # load into a json object
            generated_list = json.loads(cleaned_list_string)
            
            # Display success message
            print(f"success! :D\n")

            # print(f"Cleaned LLM Response:\n{cleaned_list_string}")
            break
            
        except Exception as e:
            print(f"An error occurred parsing the generated list json:\n {e}")
            # For testing, print response
            # print(f"Raw LLM Response:\n{response_text}\n")
            # print(f"\n\n Cleaned LLM Response:\n{response_text.strip('```json').split('```')[0].strip()}")
        
    return generated_list

### LLM Function: llm_pick_option(prompt, options, none_option) -> int
This function queries the LLM repeatedly until it chooses one of the provided options

In [7]:
import ollama

# Takes a list of dictionary objects with the 'name' and 'description'
# and builds a string with a numbered list. The none_option flag 
# inserts an "Option 0: None of these" into the choices
def build_option_list(options: list,none_option: bool=False) -> str:

    # add an option to selection none
    if none_option:
        output_string = "Option 0: None of these\n\n"
        i_offset = 1
    else:
        output_string = ""
        i_offset = 0

    # build the rest of the options from the input options list
    output_string += "\n\n".join(f"Option {i+i_offset}: {option.get('name')} to {option.get('description')}" for i, option in enumerate(options))

    return output_string

# formats the prompt and available options to choose from into a template string
def pick_option_template(prompt: str, options: str) -> str:

    output_prompt = f"""Choose which option is best suited to address the prompt.

*Prompt:* {prompt}

*Options:*\n{options}

Respond with a single number."""
    
    return output_prompt

# Asks the LLM which option (array of dictionaries with 'name' and 'description') to use to answer the prompt
# returns the LLM's choice as a single integer
# if non_option = True, choice = 0 means "None of these", otherwise 0 means the first option you provided.
# A choice = -1 means the model could not make a valid choice (it will try 100 seeds by default before giving up)
def llm_pick_option(prompt: str, options: list, none_option: bool) -> int:

    # Desired system message
    system_message = "You are a helpful AI Agent named Kiwi."

    # Convert to a string that contains the list of options
    options_list = build_option_list(options, none_option)

    # For testing, print out the options string we will send
    # print("Options List:\n\n" + options_list + "\n")

    # Embed our prompt into the pick_option_template
    prompt = pick_option_template(prompt, options_list)
    
    # For testing, print templated prompt
    # print(f"Templated Prompt:\n{prompt}")

    # set the choice to no choice selected
    choice = -1
    
    # We'll try 100 times to get a valid choice
    for seed in range(1, 101):

        # For testing, print current seed value
        # print(f"Querying model with random seed vaule {seed}...\n")

        # Send our pick tools prompt to the model
        response_text = llm_prompt(prompt, system_message)
        
        # try to convert the LLM response to an int
        try:
            choice = int(response_text)

            # For testing, displauy chosen option
            # print(f"Chose option {choice}!")
            break
            
        except:
            pass
            # For testing, display that it failed to make a choice
            # print('Failed to convert response to int :/')
        
    return choice

# Starting LLM backend

## Starting ollama server in the background

In [8]:
import subprocess
import threading
import time

def run_ollama():
    subprocess.run("ollama serve", shell=True)

ollama_thread = threading.Thread(target=run_ollama)
ollama_thread.start()

# Give Ollama some time to start up
time.sleep(10)

2025/03/04 14:35:02 routes.go:1205: INFO server config env="map[CUDA_VISIBLE_DEVICES: GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: HTTPS_PROXY: HTTP_PROXY: NO_PROXY: OLLAMA_DEBUG:false OLLAMA_FLASH_ATTENTION:false OLLAMA_GPU_OVERHEAD:0 OLLAMA_HOST:http://127.0.0.1:11434 OLLAMA_INTEL_GPU:false OLLAMA_KEEP_ALIVE:5m0s OLLAMA_KV_CACHE_TYPE: OLLAMA_LLM_LIBRARY: OLLAMA_LOAD_TIMEOUT:5m0s OLLAMA_MAX_LOADED_MODELS:0 OLLAMA_MAX_QUEUE:512 OLLAMA_MODELS:/home1/10386/lsmith9003/.ollama/models OLLAMA_MULTIUSER_CACHE:false OLLAMA_NEW_ENGINE:false OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:0 OLLAMA_ORIGINS:[http://localhost https://localhost http://localhost:* https://localhost:* http://127.0.0.1 https://127.0.0.1 http://127.0.0.1:* https://127.0.0.1:* http://0.0.0.0 https://0.0.0.0 http://0.0.0.0:* https://0.0.0.0:* app://* file://* tauri://* vscode-webview://*] OLLAMA_SCHED_SPREAD:false ROCR_VISIBLE_DEVICES: http_proxy: https_proxy: no_proxy:]"
time=2025-03

## Download the llama3.2 model

In [9]:
def download_model():
    subprocess.run("ollama run llama3.2", shell=True)

model_download_thread = threading.Thread(target=download_model)
model_download_thread.start()

[GIN] 2025/03/04 - 14:35:12 | 200 |     106.839µs |       127.0.0.1 | HEAD     "/"
[GIN] 2025/03/04 - 14:35:12 | 200 |   222.72906ms |       127.0.0.1 | POST     "/api/show"


⠙ ⠹ ⠸ ⠼ ⠴ time=2025-03-04T14:35:12.981-06:00 level=INFO source=sched.go:715 msg="new model will fit in available VRAM in single GPU, loading" model=/home1/10386/lsmith9003/.ollama/models/blobs/sha256-dde5aa3fc5ffc17176b5e8bdc82f587b24b2678c6c66101bf7da77af9f7ccdff gpu=GPU-cc7e93c2-5e50-937f-9850-671ad8808b8b parallel=4 available=16775512064 required="3.7 GiB"
⠦ ⠧ ⠇ ⠏ time=2025-03-04T14:35:13.387-06:00 level=INFO source=server.go:97 msg="system memory" total="125.6 GiB" free="117.8 GiB" free_swap="0 B"
time=2025-03-04T14:35:13.387-06:00 level=INFO source=server.go:130 msg=offload library=cuda layers.requested=-1 layers.model=29 layers.offload=29 layers.split="" memory.available="[15.6 GiB]" memory.gpu_overhead="0 B" memory.required.full="3.7 GiB" memory.required.partial="3.7 GiB" memory.required.kv="896.0 MiB" memory.required.allocations="[3.7 GiB]" memory.weights.total="2.4 GiB" memory.weights.repeating="2.1 GiB" memory.weights.nonrepeating="308.2 MiB" memory.graph.full="424.0 MiB" mem

⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ llm_load_tensors: offloading 28 repeating layers to GPU
llm_load_tensors: offloading output layer to GPU
llm_load_tensors: offloaded 29/29 layers to GPU
llm_load_tensors:   CPU_Mapped model buffer size =   308.23 MiB
llm_load_tensors:        CUDA0 model buffer size =  1918.35 MiB
⠼ ⠴ ⠦ ⠧ llama_new_context_with_model: n_seq_max     = 4
llama_new_context_with_model: n_ctx         = 8192
llama_new_context_with_model: n_ctx_per_seq = 2048
llama_new_context_with_model: n_batch       = 2048
llama_new_context_with_model: n_ubatch      = 512
llama_new_context_with_model: flash_attn    = 0
llama_new_context_with_model: freq_base     = 500000.0
llama_new_context_with_model: freq_scale    = 1
llama_new_context_with_model: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
llama_kv_cache_init: kv_size = 8192, offload = 1, type_k = 'f16', type_v = 'f16', n_layer = 28, can_shift = 1
llama_kv_cache_init:      CUDA

[GIN] 2025/03/04 - 14:35:18 | 200 |  5.839675963s |       127.0.0.1 | POST     "/api/generate"


## Check ollama server status

In [10]:
! ollama ps

[GIN] 2025/03/04 - 14:35:28 | 200 |      25.266µs |       127.0.0.1 | HEAD     "/"
[GIN] 2025/03/04 - 14:35:28 | 200 |     183.677µs |       127.0.0.1 | GET      "/api/ps"
NAME               ID              SIZE      PROCESSOR    UNTIL              
llama3.2:latest    a80c4f17acd5    4.0 GB    100% GPU     4 minutes from now    


## Close ollama server
If you want to close the server, run the cell below

In [6]:
! kill $(pgrep ollama)

# LLM Function Usage Examples

### Example sending simple prompt
The following example sends the *user_prompt* to our llm and then prints its response to the console.  The default model is set to be llama3.2, if you'd like to change it, you can add the optional input argument model="model name" to the **llm_prompt()** function like:

    response_text = llm_prompt(user_prompt,model="qwen:0.5b")

The available models are listed at https://ollama.com/library.

In [11]:
# user prompt
user_prompt = "Write a haiku about vines"

response_text = llm_prompt(user_prompt)

print(f"LLM Response:\n{response_text}\n")

llama_model_loader: loaded meta data with 30 key-value pairs and 255 tensors from /home1/10386/lsmith9003/.ollama/models/blobs/sha256-dde5aa3fc5ffc17176b5e8bdc82f587b24b2678c6c66101bf7da77af9f7ccdff (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Llama 3.2 3B Instruct
llama_model_loader: - kv   3:                           general.finetune str              = Instruct
llama_model_loader: - kv   4:                           general.basename str              = Llama-3.2
llama_model_loader: - kv   5:                         general.size_label str              = 3B
llama_model_loader: - kv   6:                               general.

[GIN] 2025/03/04 - 14:35:35 | 200 |  2.592167805s |       127.0.0.1 | POST     "/api/chat"
LLM Response:
Twisted, climbing high
Vines entwine with gentle touch
Summer's gentle kiss



### Example of asking the LLM to pick between options
In this example we will ask the LLM to decide which option (in this case, each option is a description of one of our tools) is best suited to address the given prompt. The *none_option=True* means that a response of 0 will mean the LLM chose "none of the above", and then numbers 1-N correspond to a choice of each of our N tools. Our *user_prompt* is clearly outlining a math problem, so we should expect the LLM to chose option 3 which corresponds to our **do_math** function.  For redundancy, we'll have the model chose several times to see if it's option choice varies.

In [12]:
import ollama
import json

# user prompt
user_prompt = "Multiply 626183 with 182731"

# For testing, generate function descriptions
function_descriptions = [
    generate_function_description(get_current_weather),
    generate_function_description(get_current_time),
    generate_function_description(do_math),
    generate_function_description(get_duckduckgo_result)
]       

# Pull out the name and description dictionaries from our tools json data
func_data = [func['function'] for func in function_descriptions]

# Ask LLM to choose 3 times
choices = [llm_pick_option(user_prompt, func_data, True) for i in range(0,3)]

print(f"The LLM chose options {choices}")


[GIN] 2025/03/04 - 14:35:42 | 200 |  957.969379ms |       127.0.0.1 | POST     "/api/chat"
[GIN] 2025/03/04 - 14:35:42 | 200 |  170.432633ms |       127.0.0.1 | POST     "/api/chat"
[GIN] 2025/03/04 - 14:35:43 | 200 |  384.092409ms |       127.0.0.1 | POST     "/api/chat"
[GIN] 2025/03/04 - 14:35:43 | 200 |  452.705359ms |       127.0.0.1 | POST     "/api/chat"
[GIN] 2025/03/04 - 14:35:43 | 200 |  284.075433ms |       127.0.0.1 | POST     "/api/chat"
[GIN] 2025/03/04 - 14:35:44 | 200 |  396.122293ms |       127.0.0.1 | POST     "/api/chat"
[GIN] 2025/03/04 - 14:35:44 | 200 |  150.556937ms |       127.0.0.1 | POST     "/api/chat"
[GIN] 2025/03/04 - 14:35:44 | 200 |  105.244762ms |       127.0.0.1 | POST     "/api/chat"
[GIN] 2025/03/04 - 14:35:44 | 200 |  304.161716ms |       127.0.0.1 | POST     "/api/chat"
[GIN] 2025/03/04 - 14:35:45 | 200 |  105.928413ms |       127.0.0.1 | POST     "/api/chat"
[GIN] 2025/03/04 - 14:35:45 | 200 |  543.768465ms |       127.0.0.1 | POST     "/api/chat"

### Example of asking LLM to create a list
In this example we'll prompt the LLM to create a step by step plan to accomplish a goal.  This can be very useful when trying to set up an AI agent to perform autonomously on complex multi-step problems. The **llm_create_list** functon prepends text to our prompt to instruct the LLM to respond in a specific format that we can parse into a json object. Note that the LLM often makes typos in its response which causes our fairly naive string parsing to fail.  We will try to get a well formated list 10 times before the **llm_create_list** function gives up.  Try changing the goal in the *user_prompt* string yourself to see what the LLM is capable of planning for!

In [13]:
# prompt for the model
user_prompt = """Break down the the following goal into subgoals

Goal: Create a step by step guide to writing a compelling poem."""

# send the prompt to the LLM and have it return a list
# The list should be a dict object with fields:
#    {'list_description': 'Describe list contents here',
#     'content':[{'name': 'Name of list item 1', 'description', 'Description of list item 1'},...]}
generated_list = llm_create_list(user_prompt)

# Check if list is empty, if not, print out the names of the list items
if bool(generated_list):
    # pull out list names
    list_names = [item['name'] for item in generated_list['content']]

    # print list to console for viewing
    nl = "\n"
    print(f"Generated List:\n{nl.join([f'{i+1}. {name}' for i, name in enumerate(list_names)])}\n")
else:
    print(f"No List generated :(")

trying to create list [1/10] times...[GIN] 2025/03/04 - 14:36:03 | 200 |  4.527594282s |       127.0.0.1 | POST     "/api/chat"
success! :D

Generated List:
1. Identify the theme or topic
2. Develop a unique perspective or voice
3. Choose a suitable poetic form
4. Create a compelling title
5. Use sensory language and imagery
6. Play with sound devices and rhythm
7. Use figurative language effectively
8. Edit and revise the poem
9. Get feedback from others
10. Finalize and polish the poem



# AI Agent Demo


Now we're ready to combine the functionality we've developed to create a simple AI agent!

Usage: enter the goal for the AI Agent in the *user_prompt* and the agent will
1) Create a step by step plan to achieve it based on the provided *tools*
2) Execute each step sequentially by performing a tool call
3) Concatenate all of the step results and format a final response

Note that the agent will always write a tool call for every step, even if none of the tools available are applicable. In this case there isn't a tool for "write poem" so the agent will likely do something silly like **do_math** for a step that requires text generation.  Try changing the *user_prompt* to see what kinds of tasks the agent can handle! You can also add your own functions to the cell below.  Be sure to include a docstring defining what the funciton does and also add the function to the *tools* list below.

In [16]:
# user prompt
user_prompt = "Add the temperature in Austin, TX to the temperature in Ann Arbor, MI and then write a poem whose rhyming scheme is based off the pronunciation of the combined temperatures"
print(f"User Prompt: {user_prompt}\n")

# create a list of the available tools
tools = [
    generate_function_description(get_current_weather),
    generate_function_description(get_current_time),
    generate_function_description(do_math),
    generate_function_description(get_duckduckgo_result),
]
functions = [f["function"]["name"] for f in tools]
nl = "\n"
print(f"Tools the LLM has access to:\n{nl.join([f'{f}' for f in functions])}\n")

# Format our goal and tool descriptions into a step-by-step (sbs) guide template
sbs_guide_template = f"""\nGoal: {user_prompt}.

Tools you can use:
{tools}

Break up your goal into subgoals that utilize these tools."""

print('Generating step by step guide...')

# create the step by step guide template
sbs_guide = llm_create_list(sbs_guide_template)

# string to hold numbered list with step by step guide names
sbs_guide_names = ""

# Print guide to console or quit if we failed to make one
if bool(sbs_guide):
    # pull out list names
    step_names = [item['name'] for item in sbs_guide['content']]

    # print list to console for viewing
    sbs_guide_names = f"{nl.join([f'{i+1}. {name}' for i, name in enumerate(step_names)])}\n"
    print("Generated Step by Step Guide:\n" + sbs_guide_names)
else:
    print(f"No guide generated :(")
    sys.exit()

# string array to hold responses from each step
subgoal_results = []

# Run tool calls on each step
for i, step in enumerate(sbs_guide['content']):

    print(f"Executing step [{i+1}/{len(sbs_guide['content'])}]\n")
    
    # formate subgoal prompt template
    subgoal_prompt_template = f"""Goal: {user_prompt}.

Step by step guide:
{sbs_guide_names}
"""

    # print(subgoal_prompt_template)
    
    if i>0:
        # build formated string of previous sub goal results      
        past_results = "\n".join([f"""Step {step_id+1}: {sbs_guide['content'][step_id]['name']}
    {subgoal_results[step_id]}""" for step_id in range(0,i)])

        # add to our prompt template
        subgoal_prompt_template+= "Results of previous steps:\n\n" + past_results + "\n\n"

    # add the final instruction to our prompt template
    subgoal_prompt_template+=f"""Complete the current step: {step['name']}
{step['description']}
"""   

    # print(f"Subgoal Template {i+1}:\n{subgoal_prompt_template}")
    
    # get new subgoal results, attempt 10 times if we fail
    for attempt in range(10):
        try:
            subgoal_results.append(llm_prompt_tool(subgoal_prompt_template,tools))
            break
        except:
            if attempt == 9:
                subgoal_results.append("Failed to execute tool.")

    # print(f"\nResponse to step {i+1}:\n{subgoal_results[i]}\n\n")

# write out the results of each subgoal
final_subgoal_results = "\n".join([f"""Step {step_id+1}: {sbs_guide['content'][step_id]['name']}
    {subgoal_results[step_id]}""" for step_id in range(len(subgoal_results))])

# formate final prompt template
final_prompt_template = f"""Goal: {user_prompt}.

Step by step guide:
{sbs_guide_names}

Results from each step:
{final_subgoal_results}

Complete the goal. Respond only with the answer.
"""

# perform final prompt call
response = llm_prompt(final_prompt_template)
print("====================== Final Prompt Template ======================\n")
print(final_prompt_template)
print("===================================================================\n\n")

print(f"Final Response:\n{response}")

User Prompt: Add the temperature in Austin, TX to the temperature in Ann Arbor, MI and then write a poem whose rhyming scheme is based off the pronunciation of the combined temperatures

Tools the LLM has access to:
get_current_weather
get_current_time
do_math
get_duckduckgo_result

Generating step by step guide...
trying to create list [1/10] times...[GIN] 2025/03/04 - 14:36:57 | 200 |    3.2448209s |       127.0.0.1 | POST     "/api/chat"
success! :D

Generated Step by Step Guide:
1. Get current temperature in Austin, TX
2. Get current temperature in Ann Arbor, MI
3. Add the temperatures of Austin and Ann Arbor
4. Get current time in degrees Celsius
5. Generate a poem based on the combined temperature's pronunciation

Executing step [1/5]

[GIN] 2025/03/04 - 14:36:57 | 200 |  376.573315ms |       127.0.0.1 | POST     "/api/chat"
Tool call:
[ToolCall(function=Function(name='get_current_weather', arguments={'city': 'Austin, TX'}))]

Executing step [2/5]

[GIN] 2025/03/04 - 14:36:58 | 2